In [1]:
#classificaation-YOLOV5

In [ ]:
#Load model

In [5]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

c:\Users\pkathi\Desktop\object detection\yolov5\yolov5


Cloning into 'yolov5'...


Defaulting to user installation because normal site-packages is not writeable


In [6]:
import torch

# Load YOLOv5 model (change 'yolov5s' to your custom model if you have one)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\pkathi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-7-16 Python-3.10.0 torch-2.3.1+cpu CPU

100%|██████████| 14.1M/14.1M [00:01<00:00, 7.94MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [7]:
import cv2
import numpy as np
import os
from multiprocessing.pool import ThreadPool

In [11]:
import cv2
import numpy as np
import os
from multiprocessing.pool import ThreadPool
import torch

def classify_abnormality(frame, model):
    results = model(frame)
    labels = results.pandas().xyxy[0]['name'].tolist()
    return labels

def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_dir = os.path.join(output_dir, f"abnormality_{abnormality_group_id}")
        if not os.path.exists(abnormality_dir):
            os.makedirs(abnormality_dir)

        output_frame_path = os.path.join(abnormality_dir, f"frame_{frame_number}.jpg")
        cv2.imwrite(output_frame_path, frame_resized)
        
        # Classify the abnormality
        labels = classify_abnormality(frame_resized, model)
        with open(os.path.join(abnormality_dir, "labels.txt"), "a") as f:
            f.write(f"Frame {frame_number}: {labels}\n")

    return detected, frame_gray

def motion_detection(video_path, output_dir, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Load YOLOv5 model

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1
                    os.makedirs(os.path.join(output_dir, f"abnormality_{abnormality_group_id}"))

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path = r"C:\Users\pkathi\Desktop\my work\computervision-video\output48.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/abclassification1'
abnormality_times = motion_detection(video_path, output_dir, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")


Using cache found in C:\Users\pkathi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-7-16 Python-3.10.0 torch-2.3.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Abnormality 1 start time: 0 minutes 45.00 seconds
Abnormality 1 duration: 0 minutes 12.00 seconds
Abnormality 1 start time: 0 minutes 57.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Abnormality 1 start time: 0 minutes 57.80 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Abnormality 1 start time: 0 minutes 58.20 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Total video time: 0 minutes 60.00 seconds
Abnormality detected at frame numbers: [1350, 1722, 1734, 1746, 1758]


In [ ]:
#Train with your data set

In [10]:
!python train.py --img 640 --batch 16 --epochs 10 --data C:/Users/pkathi/Desktop/data2pt.yaml --weights yolov5s.pt --cache --project C:/Users/pkathi/Desktop/training_results --name experiment_2

train: weights=yolov5s.pt, cfg=, data=C:/Users/pkathi/Desktop/data2pt.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=C:/Users/pkathi/Desktop/training_results, name=experiment_2, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 
fatal: cannot change to 'C:\Users\pkathi\Desktop\object': No such file or directory
YOLOv5  2024-7-30 Python-3.10.0 torch-2.3.1+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight

In [28]:
import cv2
import numpy as np
import os
from multiprocessing.pool import ThreadPool
import torch

def classify_abnormality(frame, model):
    results = model(frame)
    labels = results.pandas().xyxy[0]['name'].tolist()
    confidences = results.pandas().xyxy[0]['confidence'].tolist()
    return labels, confidences

def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model, confidence_threshold=0.5):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_dir = os.path.join(output_dir, f"abnormality_{abnormality_group_id}")
        if not os.path.exists(abnormality_dir):
            os.makedirs(abnormality_dir)

        output_frame_path = os.path.join(abnormality_dir, f"frame_{frame_number}.jpg")
        cv2.imwrite(output_frame_path, frame_resized)
        
        # Classify the abnormality
        labels, confidences = classify_abnormality(frame_resized, model)
        filtered_labels = [label for label, conf in zip(labels, confidences) if conf >= confidence_threshold]

        # Debug: print the frame number, labels, and confidences
        print(f"Frame {frame_number}: Labels - {labels}, Confidences - {confidences}")
        
        if filtered_labels:  # Only write if filtered_labels are detected
            with open(os.path.join(abnormality_dir, "labels.txt"), "a") as f:
                f.write(f"Frame {frame_number}: {filtered_labels}\n")

    return detected, frame_gray

def motion_detection(video_path, output_dir, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1, confidence_threshold=0.5):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    # Load the trained YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='C:/Users/pkathi/Desktop/object detection/yolov5/runs/train/exp/weights/best.pt')

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model, confidence_threshold))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1
                    os.makedirs(os.path.join(output_dir, f"abnormality_{abnormality_group_id}"), exist_ok=True)

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path = r"C:\Users\pkathi\Desktop\my work\computervision-video\output48.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/abclassification6'
abnormality_times = motion_detection(video_path, output_dir, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")


Using cache found in C:\Users\pkathi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-7-16 Python-3.10.0 torch-2.3.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Frame 1350: Labels - [], Confidences - []
Frame 1356: Labels - [], Confidences - []
Frame 1362: Labels - [], Confidences - []
Frame 1368: Labels - [], Confidences - []
Frame 1374: Labels - [], Confidences - []
Frame 1380: Labels - [], Confidences - []
Frame 1386: Labels - [], Confidences - []
Frame 1392: Labels - [], Confidences - []
Frame 1398: Labels - [], Confidences - []
Frame 1404: Labels - [], Confidences - []
Frame 1410: Labels - [], Confidences - []
Frame 1416: Labels - [], Confidences - []
Frame 1422: Labels - [], Confidences - []
Frame 1428: Labels - [], Confidences - []
Frame 1434: Labels - [], Confidences - []
Frame 1440: Labels - [], Confidences - []
Frame 1446: Labels - [], Confidences - []
Frame 1452: Labels - [], Confidences - []
Frame 1458: Labels - [], Confidences - []
Frame 1464: Labels - [], Confidences - []
Frame 1470: Labels - [], Confidences - []
Frame 1476: Labels - [], Confidences - []
Frame 1482: Labels - [], Confidences - []
Frame 1488: Labels - [], Confidenc

In [15]:
import cv2
import numpy as np
import os
from multiprocessing.pool import ThreadPool
import torch

def classify_abnormality(frame, model):
    results = model(frame)
    labels = results.pandas().xyxy[0]['name'].tolist()
    return labels

def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_dir = os.path.join(output_dir, f"abnormality_{abnormality_group_id}")
        if not os.path.exists(abnormality_dir):
            os.makedirs(abnormality_dir)

        output_frame_path = os.path.join(abnormality_dir, f"frame_{frame_number}.jpg")
        cv2.imwrite(output_frame_path, frame_resized)
        
        # Classify the abnormality
        labels = classify_abnormality(frame_resized, model)
        with open(os.path.join(abnormality_dir, "labels.txt"), "a") as f:
            f.write(f"Frame {frame_number}: {labels}\n")

    return detected, frame_gray

def motion_detection(video_path, output_dir, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    # Load the custom YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='C:/Users/pkathi/Desktop/object detection/yolov5/yolov5s.pt')

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1
                    os.makedirs(os.path.join(output_dir, f"abnormality_{abnormality_group_id}"))

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path = r"C:\Users\pkathi\Desktop\my work\computervision-video\output48.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/abclassification2'
abnormality_times = motion_detection(video_path, output_dir, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")


Using cache found in C:\Users\pkathi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-7-16 Python-3.10.0 torch-2.3.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Abnormality 1 start time: 0 minutes 45.00 seconds
Abnormality 1 duration: 0 minutes 12.00 seconds
Abnormality 1 start time: 0 minutes 57.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Abnormality 1 start time: 0 minutes 57.80 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Abnormality 1 start time: 0 minutes 58.20 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Total video time: 0 minutes 60.00 seconds
Abnormality detected at frame numbers: [1350, 1722, 1734, 1746, 1758]


In [17]:
import cv2
import numpy as np
import os
from multiprocessing.pool import ThreadPool
import torch

def classify_abnormality(frame, model):
    results = model(frame)
    labels = results.pandas().xyxy[0]['name'].tolist()
    return labels

def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_dir = os.path.join(output_dir, f"abnormality_{abnormality_group_id}")
        if not os.path.exists(abnormality_dir):
            os.makedirs(abnormality_dir)

        output_frame_path = os.path.join(abnormality_dir, f"frame_{frame_number}.jpg")
        cv2.imwrite(output_frame_path, frame_resized)
        
        # Classify the abnormality
        labels = classify_abnormality(frame_resized, model)
        with open(os.path.join(abnormality_dir, "labels.txt"), "a") as f:
            f.write(f"Frame {frame_number}: {labels}\n")

    return detected, frame_gray

def motion_detection(video_path, output_dir, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    # Load the custom YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='C:/Users/pkathi/Desktop/object detection/yolov5/yolov5s.pt')

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1
                    os.makedirs(os.path.join(output_dir, f"abnormality_{abnormality_group_id}"))

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path = r"C:\Users\pkathi\Desktop\my work\computervision-video\output48.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/abclassification3'
abnormality_times = motion_detection(video_path, output_dir, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")


Using cache found in C:\Users\pkathi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-7-16 Python-3.10.0 torch-2.3.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Abnormality 1 start time: 0 minutes 45.00 seconds
Abnormality 1 duration: 0 minutes 12.00 seconds
Abnormality 1 start time: 0 minutes 57.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Abnormality 1 start time: 0 minutes 57.80 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Abnormality 1 start time: 0 minutes 58.20 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Total video time: 0 minutes 60.00 seconds
Abnormality detected at frame numbers: [1350, 1722, 1734, 1746, 1758]


In [21]:
import torch

# Load the custom YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='C:/Users/pkathi/Desktop/object detection/yolov5/runs/train/exp/weights/best.pt')
print(model.names)  # Print out the recognized labels

Using cache found in C:\Users\pkathi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-7-16 Python-3.10.0 torch-2.3.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


{0: 'fire', 1: 'smoke', 2: 'sparkle'}


In [23]:
import cv2
import numpy as np
import os
from multiprocessing.pool import ThreadPool
import torch

def classify_abnormality(frame, model):
    results = model(frame)
    labels = results.pandas().xyxy[0]['name'].tolist()
    return labels

def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_dir = os.path.join(output_dir, f"abnormality_{abnormality_group_id}")
        if not os.path.exists(abnormality_dir):
            os.makedirs(abnormality_dir)

        output_frame_path = os.path.join(abnormality_dir, f"frame_{frame_number}.jpg")
        cv2.imwrite(output_frame_path, frame_resized)
        
        # Classify the abnormality
        labels = classify_abnormality(frame_resized, model)
        with open(os.path.join(abnormality_dir, "labels.txt"), "a") as f:
            f.write(f"Frame {frame_number}: {labels}\n")

    return detected, frame_gray

def motion_detection(video_path, output_dir, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    # Load the custom YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='C:/Users/pkathi/Desktop/object detection/yolov5/runs/train/exp/weights/best.pt')

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1
                    os.makedirs(os.path.join(output_dir, f"abnormality_{abnormality_group_id}"))

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path = r"C:\Users\pkathi\Desktop\my work\computervision-video\output48.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/abclassification4'
abnormality_times = motion_detection(video_path, output_dir, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")


Using cache found in C:\Users\pkathi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-7-16 Python-3.10.0 torch-2.3.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Abnormality 1 start time: 0 minutes 45.00 seconds
Abnormality 1 duration: 0 minutes 12.00 seconds
Abnormality 1 start time: 0 minutes 57.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Abnormality 1 start time: 0 minutes 57.80 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Abnormality 1 start time: 0 minutes 58.20 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Total video time: 0 minutes 60.00 seconds
Abnormality detected at frame numbers: [1350, 1722, 1734, 1746, 1758]


In [25]:
import cv2
import numpy as np
import os
from multiprocessing.pool import ThreadPool
import torch

def classify_abnormality(frame, model):
    results = model(frame)
    labels = results.pandas().xyxy[0]['name'].tolist()
    return labels

def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_dir = os.path.join(output_dir, f"abnormality_{abnormality_group_id}")
        if not os.path.exists(abnormality_dir):
            os.makedirs(abnormality_dir)

        output_frame_path = os.path.join(abnormality_dir, f"frame_{frame_number}.jpg")
        cv2.imwrite(output_frame_path, frame_resized)
        
        # Classify the abnormality
        labels = classify_abnormality(frame_resized, model)
        with open(os.path.join(abnormality_dir, "labels.txt"), "a") as f:
            f.write(f"Frame {frame_number}: {labels}\n")

    return detected, frame_gray

def motion_detection(video_path, output_dir, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    # Load the custom YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='C:/Users/pkathi/Desktop/object detection/yolov5/runs/train/exp/weights/best.pt')

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1
                    os.makedirs(os.path.join(output_dir, f"abnormality_{abnormality_group_id}"))

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path = r"C:\Users\pkathi\Desktop\my work\computervision-video\C1007trimmed116-118.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/abclassification5'
abnormality_times = motion_detection(video_path, output_dir, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")


Using cache found in C:\Users\pkathi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-7-16 Python-3.10.0 torch-2.3.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Abnormality 1 start time: 0 minutes 57.60 seconds
Abnormality 1 duration: 0 minutes 0.80 seconds
Abnormality 1 start time: 0 minutes 58.60 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Abnormality 1 start time: 1 minutes 3.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Abnormality 1 start time: 1 minutes 3.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Abnormality 1 start time: 1 minutes 17.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Abnormality 1 start time: 1 minutes 17.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Total video time: 1 minutes 60.00 seconds
Abnormality detected at frame numbers: [1728, 1758, 1890, 1902, 2310, 2322]


In [29]:
import cv2
import numpy as np
import os
from multiprocessing.pool import ThreadPool
import torch

def classify_abnormality(frame, model, confidence_threshold=0.5):
    results = model(frame)
    labels = results.pandas().xyxy[0]['name'].tolist()
    confidences = results.pandas().xyxy[0]['confidence'].tolist()
    filtered_labels = [label for label, conf in zip(labels, confidences) if conf >= confidence_threshold]
    return filtered_labels

def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model, confidence_threshold=0.5):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_dir = os.path.join(output_dir, f"abnormality_{abnormality_group_id}")
        if not os.path.exists(abnormality_dir):
            os.makedirs(abnormality_dir)

        output_frame_path = os.path.join(abnormality_dir, f"frame_{frame_number}.jpg")
        cv2.imwrite(output_frame_path, frame_resized)
        
        # Classify the abnormality
        labels = classify_abnormality(frame_resized, model, confidence_threshold)

        # Debug: print the frame number, labels
        print(f"Frame {frame_number}: {labels}")
        
        if labels:  # Only write if labels are detected
            with open(os.path.join(abnormality_dir, "labels.txt"), "a") as f:
                f.write(f"Frame {frame_number}: {labels}\n")

    return detected, frame_gray

def motion_detection(video_path, output_dir, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1, confidence_threshold=0.5):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    # Load the trained YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='C:/Users/pkathi/Desktop/object detection/yolov5/runs/train/exp/weights/best.pt')

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model, confidence_threshold))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1
                    os.makedirs(os.path.join(output_dir, f"abnormality_{abnormality_group_id}"), exist_ok=True)

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path = r"C:\Users\pkathi\Desktop\my work\computervision-video\output48.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/abclassification7'
abnormality_times = motion_detection(video_path, output_dir, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")


Using cache found in C:\Users\pkathi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-7-16 Python-3.10.0 torch-2.3.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Frame 1350: []
Frame 1356: []
Frame 1362: []
Frame 1368: []
Frame 1374: []
Frame 1380: []
Frame 1386: []
Frame 1392: []
Frame 1398: []
Frame 1404: []
Frame 1410: []
Frame 1416: []
Frame 1422: []
Frame 1428: []
Frame 1434: []
Frame 1440: []
Frame 1446: []
Frame 1452: []
Frame 1458: []
Frame 1464: []
Frame 1470: []
Frame 1476: []
Frame 1482: []
Frame 1488: []
Frame 1494: []
Frame 1500: []
Frame 1506: []
Frame 1512: []
Frame 1518: []
Frame 1524: []
Frame 1530: []
Frame 1536: []
Frame 1542: []
Frame 1548: []
Frame 1554: []
Frame 1560: []
Frame 1566: []
Frame 1572: []
Frame 1578: []
Frame 1584: []
Frame 1590: []
Frame 1596: []
Frame 1602: []
Frame 1608: []
Frame 1614: []
Frame 1620: []
Frame 1626: []
Frame 1632: []
Frame 1638: []
Frame 1644: []
Frame 1650: []
Frame 1656: []
Frame 1662: []
Frame 1668: []
Frame 1674: []
Frame 1680: []
Frame 1686: []
Frame 1692: []
Frame 1698: []
Frame 1704: []
Abnormality 1 start time: 0 minutes 45.00 seconds
Abnormality 1 duration: 0 minutes 12.00 seconds
Fr

In [1]:
import cv2
import numpy as np
import os
from multiprocessing.pool import ThreadPool
import torch

def classify_abnormality(frame, model, confidence_threshold=0.5):
    results = model(frame)
    labels = results.pandas().xyxy[0]['name'].tolist()
    confidences = results.pandas().xyxy[0]['confidence'].tolist()
    filtered_labels = [label for label, conf in zip(labels, confidences) if conf >= confidence_threshold]
    return filtered_labels

def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model, confidence_threshold=0.5):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_dir = os.path.join(output_dir, f"abnormality_{abnormality_group_id}")
        if not os.path.exists(abnormality_dir):
            os.makedirs(abnormality_dir)

        output_frame_path = os.path.join(abnormality_dir, f"frame_{frame_number}.jpg")
        cv2.imwrite(output_frame_path, frame_resized)
        
        # Classify the abnormality
        labels = classify_abnormality(frame_resized, model, confidence_threshold)

        # Debug: print the frame number, labels
        print(f"Frame {frame_number}: {labels}")
        
        if labels:  # Only write if labels are detected
            with open(os.path.join(abnormality_dir, "labels.txt"), "a") as f:
                f.write(f"Frame {frame_number}: {labels}\n")

    return detected, frame_gray

def motion_detection(video_path, output_dir, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1, confidence_threshold=0.5):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    # Load the trained YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='C:/Users/pkathi/Desktop/object detection/yolov5/runs/train/exp/weights/best.pt')

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model, confidence_threshold))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1
                    os.makedirs(os.path.join(output_dir, f"abnormality_{abnormality_group_id}"), exist_ok=True)

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path =  r"C:\Users\pkathi\Desktop\my work\computervision-video\C1007trimmed116-118.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/abclastest1'
abnormality_times = motion_detection(video_path, output_dir, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")


Using cache found in C:\Users\pkathi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-7-16 Python-3.10.0 torch-2.3.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Frame 1728: []
Frame 1734: []
Frame 1740: []
Frame 1746: []
Abnormality 1 start time: 0 minutes 57.60 seconds
Abnormality 1 duration: 0 minutes 0.80 seconds
Frame 1758: []
Abnormality 1 start time: 0 minutes 58.60 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Frame 1890: []
Abnormality 1 start time: 1 minutes 3.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Frame 1902: []
Abnormality 1 start time: 1 minutes 3.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Frame 2310: []
Abnormality 1 start time: 1 minutes 17.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Frame 2322: []
Abnormality 1 start time: 1 minutes 17.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Total video time: 1 minutes 60.00 seconds
Abnormality detected at frame numbers: [1728, 1758, 1890, 1902, 2310, 2322]


In [2]:
import cv2
import numpy as np
import os
from multiprocessing.pool import ThreadPool
import torch

def classify_abnormality(frame, model, confidence_threshold=0.5):
    results = model(frame)
    labels = results.pandas().xyxy[0]['name'].tolist()
    confidences = results.pandas().xyxy[0]['confidence'].tolist()
    filtered_labels = [label for label, conf in zip(labels, confidences) if conf >= confidence_threshold]
    return filtered_labels

def process_frame(frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model, confidence_threshold=0.5):
    frame_resized = cv2.resize(frame, (0, 0), fx=resize_factor, fy=resize_factor)
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(prev_frame_gray, frame_gray)
    _, thresh = cv2.threshold(frame_diff, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected = any(cv2.contourArea(contour) > min_contour_area for contour in contours)
    if detected:
        abnormality_dir = os.path.join(output_dir, f"abnormality_{abnormality_group_id}")
        if not os.path.exists(abnormality_dir):
            os.makedirs(abnormality_dir)

        output_frame_path = os.path.join(abnormality_dir, f"frame_{frame_number}.jpg")
        cv2.imwrite(output_frame_path, frame_resized)
        
        # Classify the abnormality
        labels = classify_abnormality(frame_resized, model, confidence_threshold)

        # Debug: print the frame number, labels
        print(f"Frame {frame_number}: {labels}")
        
        if labels:  # Only write if labels are detected
            with open(os.path.join(abnormality_dir, "labels.txt"), "a") as f:
                f.write(f"Frame {frame_number}: {labels}\n")

    return detected, frame_gray

def motion_detection(video_path, output_dir, threshold_value=25, min_contour_area=500, resize_factor=0.5, minimal_fps=5, min_abnormality_gap_minutes=1, confidence_threshold=0.5):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_skip_interval = int(round(original_fps / minimal_fps))

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading the first frame")
        return

    prev_frame = cv2.resize(prev_frame, (0, 0), fx=resize_factor, fy=resize_factor)
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    abnormality_times = []

    frame_number = 0
    abnormality_detected = False
    abnormality_start_frame = None
    abnormality_group_id = 0
    last_abnormality_time = None

    os.makedirs(output_dir, exist_ok=True)

    pool = ThreadPool(processes=4)  # Use a thread pool with 4 threads

    # Load the trained YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='C:/Users/pkathi/Desktop/object detection/yolov5/runs/train/exp/weights/best.pt')

    while True:
        for _ in range(frame_skip_interval):
            ret = cap.grab()  # Use grab to skip frames efficiently
            if not ret:
                break
            frame_number += 1

        if not ret:
            break

        ret, frame = cap.retrieve()  # Retrieve the frame after skipping
        if not ret:
            break

        # Use thread pool to process frame
        result = pool.apply_async(process_frame, (frame, prev_frame_gray, threshold_value, min_contour_area, original_fps, frame_number, resize_factor, output_dir, abnormality_group_id, model, confidence_threshold))
        detected, prev_frame_gray = result.get()

        if detected:
            current_abnormality_time = frame_number / original_fps

            if not abnormality_detected:
                if last_abnormality_time is None or (current_abnormality_time - last_abnormality_time) >= (min_abnormality_gap_minutes * 60):
                    abnormality_group_id += 1
                    os.makedirs(os.path.join(output_dir, f"abnormality_{abnormality_group_id}"), exist_ok=True)

                abnormality_start_frame = frame_number
                abnormality_detected = True
                abnormality_times.append(frame_number)
                last_abnormality_time = current_abnormality_time

        else:
            if abnormality_detected:
                abnormality_detected = False
                abnormality_end_frame = frame_number
                abnormality_start_seconds = abnormality_start_frame / original_fps
                abnormality_end_seconds = abnormality_end_frame / original_fps
                abnormality_duration_seconds = abnormality_end_seconds - abnormality_start_seconds

                print(f"Abnormality {abnormality_group_id} start time: {int(abnormality_start_seconds // 60)} minutes {abnormality_start_seconds % 60:.2f} seconds")
                print(f"Abnormality {abnormality_group_id} duration: {int(abnormality_duration_seconds // 60)} minutes {abnormality_duration_seconds % 60:.2f} seconds")

    cap.release()
    pool.close()
    pool.join()

    total_time_seconds = frame_number / original_fps
    print(f"Total video time: {int(total_time_seconds // 60)} minutes {total_time_seconds % 60:.2f} seconds")

    return abnormality_times

# Example usage
video_path =  r"C:\Users\pkathi\Desktop\my work\computervision-video\C1007trimmed116-118.mp4"
output_dir = 'C:/Users/pkathi/Desktop/my work/computervision-video/abclastest2'
abnormality_times = motion_detection(video_path, output_dir, minimal_fps=5)

if abnormality_times:
    print("Abnormality detected at frame numbers:", abnormality_times)
else:
    print("No abnormality detected in the video.")


Using cache found in C:\Users\pkathi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-7-16 Python-3.10.0 torch-2.3.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Frame 1728: []
Frame 1734: []
Frame 1740: []
Frame 1746: []
Abnormality 1 start time: 0 minutes 57.60 seconds
Abnormality 1 duration: 0 minutes 0.80 seconds
Frame 1758: []
Abnormality 1 start time: 0 minutes 58.60 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Frame 1890: []
Abnormality 1 start time: 1 minutes 3.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Frame 1902: []
Abnormality 1 start time: 1 minutes 3.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Frame 2310: []
Abnormality 1 start time: 1 minutes 17.00 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Frame 2322: []
Abnormality 1 start time: 1 minutes 17.40 seconds
Abnormality 1 duration: 0 minutes 0.20 seconds
Total video time: 1 minutes 60.00 seconds
Abnormality detected at frame numbers: [1728, 1758, 1890, 1902, 2310, 2322]
